In [1]:
# !pip install transformers

In [2]:
import warnings
from tqdm.auto import tqdm
warnings.filterwarnings('ignore')  # Ignore all warnings
tqdm.pandas()

# LLM Example

Today we'll see how to work with decoder models in the zero-shot mode. We'll start with the basic GPT3 zero-shot example and then switch to more advanced LLMs.

In [3]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
device

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 3070 Laptop GPU


device(type='cuda')

## ruGPT3 example

Load [ruGPT3](https://huggingface.co/ai-forever/rugpt3large_based_on_gpt2).

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("ai-forever/rugpt3large_based_on_gpt2")
model = AutoModelForCausalLM.from_pretrained("ai-forever/rugpt3large_based_on_gpt2")

In [5]:
model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1536)
    (wpe): Embedding(2048, 1536)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=4608, nx=1536)
          (c_proj): Conv1D(nf=1536, nx=1536)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=6144, nx=1536)
          (c_proj): Conv1D(nf=1536, nx=6144)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1536, out_features=50257, bias=False)
)

# Zero-shot

We use model loss for the zero-shot classification.

GPT-based models utilize per-token cross-entropy
loss, which is reduced to negative log probability
due to one-hot encoding of the tokens. **The idea is to select the target label associated with the prompt that results in the lowest sum of negative log probabilities for its tokens.**



In [6]:
import math
def get_loss_num(text):
    # Tokenize the input text and move it to the specified device
    inputs = tokenizer(text, return_tensors="pt").to(device)

    # Shift the inputs to create labels for the next-token prediction task
    labels = inputs["input_ids"].clone()

    # Move labels to the correct device if you're using GPU
    labels = labels.to(device)

    # Calculate loss
    outputs = model(**inputs, labels=labels, use_cache=True)  # Add use_cache=True to use the cache class
    loss = outputs.loss
    return loss.item()

def clean(text):
    text = re.sub(r'\((\d+)\)', '', text)
    return text

### Task: twitter tone analysis

Today we'll solve a sentiment analysis task. Let us start with some toy examples and try to come up with the prompts that can distinguish positive and negative texts.

**Positive promt example**

In [7]:
text = 'жизнь отличная'
get_loss_num('Позитивный твит: ' + text)

6.2020111083984375

**Negative prompt example**

In [8]:
get_loss_num('Негативный твит: ' + text)

7.345581531524658

Let's add smiles!

In [9]:
print(get_loss_num('Позитивный твит: ' + text + ')))'))
print(get_loss_num('Негативный твит: ' + text + '((('))

6.151878833770752
7.050114154815674


Now we implement a function that selects the label which yeilds the lowest loss.

In [10]:
def predict_zero_shot(text, pos = 'Позитивный твит: {})))', neg = 'Негативный твит: {}((('):
  pos_loss = get_loss_num(pos.format(text))
  neg_loss = get_loss_num(neg.format(text))
  if pos_loss < neg_loss:
    return 'positive'
  return 'negative'

predict_zero_shot(text)

'positive'

Let's apply this approach to the twitter sentimant classification task.

In [11]:
url = "https://drive.usercontent.google.com/download?id=17qSrjy5NyknCfhs1kqGwHcHgml9UzpvS&export=download&authuser=0&confirm=t&uuid=cb32846f-bc96-4eb0-9e29-57d27a89e369&at=AN_67v2rr2Fh_KVc0V-EDJQ7bufm:1729946024386"

In [12]:
import pandas as pd
df = pd.read_csv(url, index_col = 0)
df.head()

,text,label
0,на работе был полный пиддес :| и так каждое за...,negative
1,"Коллеги сидят рубятся в Urban terror, а я из-з...",negative
2,@elina_4post как говорят обещаного три года жд...,negative
3,"Желаю хорошего полёта и удачной посадки,я буду...",negative
4,"Обновил за каким-то лешим surf, теперь не рабо...",negative


In [13]:
df.tail()

,text,label
95,"Встречайте, мои супер одногруппницы, будущие и...",positive
96,"все,я вас покидаю,результаты гляну вечером)#би...",positive
97,RT @Dasha_crazy_69: @DashkaTeddy дыы))) но кто...,positive
98,Почти приехали в родное селенье!) @ москва-рига,positive
99,На*уй ваши Канары и Мальдивы ! Тут новая тема ...,positive


In [14]:
from sklearn.metrics import accuracy_score
df['preds'] = df.text.apply(predict_zero_shot)
accuracy_score(df.label, df.preds)

0.74

In [15]:
from sklearn.metrics import f1_score
def encode_label(x):
  if x == 'negative':
    return 0
  return 1
f1_score(df.label.apply(encode_label), df.preds.apply(encode_label))

0.7868852459016393

## QWEN2.5

[Qwen2.5](https://huggingface.co/Qwen/Qwen2.5-0.5B-Instruct) is a small LLM which can be run in Colab.

In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")
model.to(device);


First look how it works for simple text generation task.

In [17]:
text = "Продолжи поговорку:\nБез труда"
print(text)

Продолжи поговорку:
Без труда


In [18]:
tokens = tokenizer(text, add_special_tokens=True, return_tensors="pt").to(device)
tokens

{'input_ids': tensor([[ 53645,   9516,  47081,   1802,   5063,  14497, 125661,  35252,    510,
          60332,  31885,  10813,  19763,  39490]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

First try:

In [19]:
outputs = model.generate(**tokens, top_k=1).cpu()
print(tokenizer.batch_decode(outputs)[0])


Продолжи поговорку:
Без труда не выйдешь,


In [20]:
outputs = model.generate(**tokens, num_beams=4, max_length=30).cpu()
print(tokenizer.batch_decode(outputs)[0])

Продолжи поговорку:
Без труда ничего не добьешься, но без труда ничего не добь


In [21]:
outputs = model.generate(**tokens, num_beams=4, num_return_sequences=4, max_length=40).cpu()
print("\n\n\n".join(tokenizer.batch_decode(outputs)))

Продолжи поговорку:
Без труда ничего не добьешься, но без труда ничего и не добьешься.

Исправь ошибки


Продолжи поговорку:
Без труда ничего не добьешься, но без труда ничего и не добьешься.

Исправь ошибку


Продолжи поговорку:
Без труда ничего не добьешься, но без труда и ничего не добьешься.

Вот продолжение этой п


Продолжи поговорку:
Без труда ничего не добьешься, но без труда ничего и не добьешься.

Используя этот


## System prompt

A **system prompt** (or system message) is a special instruction provided to an LLM that defines its behavior, tone, personality, and constraints during interactions with users. It serves as a foundational guideline that sets expectations for how the model should respond to user inputs throughout a session.

But how? Let's ask [Mistral](https://chat.mistral.ai/), [ChatGPT](https://chatgpt.com), or Gemini! Open a model chat and type:


```
Add system prompt in gwen 2.5
```

Let's now add a system prompt!



In [22]:
system_prompt = "Ты — помощник, который генерирует пословицы на русском языке."  # Define your system prompt

prompt = "Продолжи поговорку:\nБез труда"
# Combine system prompt and user prompt into a full prompt
full_prompt = f"{system_prompt}\n\n{prompt}"
# Tokenize the full prompt
tokens = tokenizer(full_prompt, return_tensors="pt").to(device)

# Generate the response using the Qwen-2 model
outputs = model.generate(**tokens, num_beams=4, num_return_sequences=4, max_length=70).cpu()
print("\n\n\n".join([x.split('\n\n')[-1] for x in tokenizer.batch_decode(outputs)]))

Без труда не пришёл, без труда и не


1. Без труда не пришё


Без труда не пришёл, 
без труда


Без труда не пришёл, без труда не у


## Gwen2.5 for sentiment analysis

Now, let's look how it solves the sentiment analysis task. First, try the simple generation approach.



In [23]:
text = 'жизнь отличная'
prompt = "Напиши pos в случае если приведенный текст твита позитивный и neg в случае если негативный. Ничего больше не добавляй. Текст твита:\n{}".format(text)
print(prompt)
# Combine system prompt and user prompt into a full prompt
# Tokenize the full prompt
tokens = tokenizer(prompt, return_tensors="pt").to(device)

outputs = model.generate(**tokens, num_beams=2, num_return_sequences=1, max_length=100).cpu()
print(tokenizer.batch_decode(outputs)[0].replace(prompt,''))

Напиши pos в случае если приведенный текст твита позитивный и neg в случае если негативный. Ничего больше не добавляй. Текст твита:
жизнь отличная
, работа хорошая, семья счастливая, друзья веселые, планы на будущее интересные. 

pos = 1
neg = 0

pos = 1
neg = 0




Add a system prompt.

In [24]:
system_prompt = "Ты — помощник, который задачу sentiment analysis."  # Define your system prompt
text = 'жизнь отличная'

prompt = "Напиши pos в случае если приведенный текст твита позитивный и neg в случае если негативный. Ничего больше не добавляй. Текст твита:\n{}".format(text)
# Combine system prompt and user prompt into a full prompt
full_prompt = f"{system_prompt}\n\n{prompt}"
# Tokenize the full prompt
tokens = tokenizer(full_prompt, return_tensors="pt").to(device)

outputs = model.generate(**tokens, num_beams=2, num_return_sequences=1, max_length=100).cpu()
print(tokenizer.batch_decode(outputs)[0].replace(full_prompt,''))

, всё хорошо

pos, neg

pos, neg

pos, neg

pos, neg

pos, neg

pos, neg

pos, neg

pos, neg


The model is too small and the result is now that good. But what about the loss variant?

In [25]:
print(get_loss_num('Позитивный твит: ' + text))
print(get_loss_num('Негативный твит: ' + text))

3.7961068153381348
4.003836631774902


In [26]:
print(get_loss_num('Позитивный твит: ' + text + ')))'))
print(get_loss_num('Негативный твит: ' + text + '((('))

4.081406116485596
4.601904392242432


In [27]:
from sklearn.metrics import accuracy_score
df['preds_qwen'] = df.text.apply(predict_zero_shot)
accuracy_score(df.label, df.preds_qwen)

0.81

In [28]:
f1_score(df.label.apply(encode_label), df.preds_qwen.apply(encode_label))

0.8347826086956521

=======================================================================================

The task is to select the best prompt for the sentiment analysis tasks. Experiment with different prompts in the predict_zero_shot function (see the last part of the zero_shot_example.ipynb).

Compare the quality of different prompts on the Twitter sample (from twitter_short.csv). Use accuracy as a metric.

In [29]:
def get_results_with_experiments(params_list):
    results = []
    experiment_number = 1
    
    for params in params_list:
        
        pos = params["pos"]
        neg = params["neg"]
        
        print(f"\nЭксперимент {experiment_number}")
        print(f"pos: {pos}")
        print(f"neg: {neg}")
        
        df['preds_'] = df['text'].progress_apply(lambda x: predict_zero_shot(x, pos=pos, neg=neg))
        
        accuracy = accuracy_score(df['label'], df['preds_'])
        f1 = round(f1_score(df['label'].apply(encode_label), df['preds_'].apply(encode_label)), 3)
        
        print(f"Accuracy: {accuracy}")
        print(f"F1 Score: {f1}")
        
        results.append({
            'experiment_number': experiment_number,
            'pos': pos,
            'neg': neg,
            'f1_score': f1,
            'accuracy': accuracy,
        })
        
        experiment_number += 1

    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values(by='accuracy', ascending=False).reset_index(drop=True)
    torch.cuda.empty_cache()
    
    return results_df

In [30]:
exp_list = [
    {
        "pos": "Позитивный твит: {})))",
        "neg": "Негативный твит: {}((("
    },
    {
        "pos": "Позитив: {}",
        "neg": "Негатив: {}"
    },
    {
        "pos": "Позитивно)): {}",
        "neg": "Негативно((: {}"
    },
    {
        "pos": "Позитив: {}))",
        "neg": "Негатив: {}(("
    },
    {
        "pos": "Позитив: {}, хорошо, отлично, замечательно, великолепно",
        "neg": "Негатив: {}, плохо, отвратительно, ужасно, мерзко"
    },
    {
        "pos": "Позитивный твит: {}) - хорошо)))",
        "neg": "Негативный твит: {}( - плохо((("
    },
    {
        "pos": "Позитивное сообщение : {})), хорошо)))",
        "neg": "Негативное сообщение: {}((, плохо((("
    },
    {
        "pos": "Позитивное сообщение : {})))",
        "neg": "Негативное сообщение: {}((("
    },
    {
        "pos": "{})))",
        "neg": "{}((("
    },
    {
        "pos": "{}))), позитивный твит",
        "neg": "{}(((, Негативный твит"
    },
    {
        "pos": "Отлично))): {}",
        "neg": "Ужасно(((: {}"
    },
    
]

In [31]:
get_results_with_experiments(exp_list)


Эксперимент 1
pos: Позитивный твит: {})))
neg: Негативный твит: {}(((


  0%|          | 0/100 [00:00<?, ?it/s]

Accuracy: 0.81
F1 Score: 0.835

Эксперимент 2
pos: Позитив: {}
neg: Негатив: {}


  0%|          | 0/100 [00:00<?, ?it/s]

Accuracy: 0.66
F1 Score: 0.622

Эксперимент 3
pos: Позитивно)): {}
neg: Негативно((: {}


...

  0%|          | 0/100 [00:00<?, ?it/s]

Accuracy: 0.62
F1 Score: 0.725

Эксперимент 11
pos: Отлично))): {}
neg: Ужасно(((: {}


  0%|          | 0/100 [00:00<?, ?it/s]

Accuracy: 0.54
F1 Score: 0.676


,experiment_number,pos,neg,f1_score,accuracy
0,4,Позитив: {})),Негатив: {}((,0.873,0.86
1,1,Позитивный твит: {}))),Негативный твит: {}(((,0.835,0.81
2,9,{}))),{}(((,0.821,0.79
3,8,Позитивное сообщение : {}))),Негативное сообщение: {}(((,0.675,0.75
4,5,"Позитив: {}, хорошо, отлично, замечательно, ве...","Негатив: {}, плохо, отвратительно, ужасно, мерзко",0.622,0.72
5,7,"Позитивное сообщение : {})), хорошо)))","Негативное сообщение: {}((, плохо(((",0.571,0.70
6,6,Позитивный твит: {}) - хорошо))),Негативный твит: {}( - плохо(((,0.758,0.68
7,2,Позитив: {},Негатив: {},0.622,0.66
8,10,"{}))), позитивный твит","{}(((, Негативный твит",0.725,0.62
9,11,Отлично))): {},Ужасно(((: {},0.676,0.54


In [32]:
#ruGPT
tokenizer = AutoTokenizer.from_pretrained("ai-forever/rugpt3large_based_on_gpt2")
model = AutoModelForCausalLM.from_pretrained("ai-forever/rugpt3large_based_on_gpt2")
model.to(device)
None

In [33]:
get_results_with_experiments(exp_list)


Эксперимент 1
pos: Позитивный твит: {})))
neg: Негативный твит: {}(((


  0%|          | 0/100 [00:00<?, ?it/s]

Accuracy: 0.74
F1 Score: 0.787

Эксперимент 2
pos: Позитив: {}
neg: Негатив: {}


  0%|          | 0/100 [00:00<?, ?it/s]

Accuracy: 0.69
F1 Score: 0.627

Эксперимент 3
pos: Позитивно)): {}
neg: Негативно((: {}


...

  0%|          | 0/100 [00:00<?, ?it/s]

Accuracy: 0.68
F1 Score: 0.628

Эксперимент 11
pos: Отлично))): {}
neg: Ужасно(((: {}


  0%|          | 0/100 [00:00<?, ?it/s]

Accuracy: 0.5
F1 Score: 0.0


,experiment_number,pos,neg,f1_score,accuracy
0,4,Позитив: {})),Негатив: {}((,0.869,0.87
1,8,Позитивное сообщение : {}))),Негативное сообщение: {}(((,0.776,0.81
2,7,"Позитивное сообщение : {})), хорошо)))","Негативное сообщение: {}((, плохо(((",0.750,0.80
3,6,Позитивный твит: {}) - хорошо))),Негативный твит: {}( - плохо(((,0.820,0.78
4,1,Позитивный твит: {}))),Негативный твит: {}(((,0.787,0.74
5,9,{}))),{}(((,0.603,0.71
6,2,Позитив: {},Негатив: {},0.627,0.69
7,10,"{}))), позитивный твит","{}(((, Негативный твит",0.628,0.68
8,5,"Позитив: {}, хорошо, отлично, замечательно, ве...","Негатив: {}, плохо, отвратительно, ужасно, мерзко",0.148,0.54
9,3,Позитивно)): {},Негативно((: {},0.671,0.51


Эксперимент № 4 - показал наилучшие результаты для обоеих моделей.

pos = "Позитив: {}))"	neg = "Негатив: {}(("